In [3]:
import sqlite3
import pandas as pd

# Connect to the SQLite DB
conn = sqlite3.connect('db/event_history.db')

# Load the table into a DataFrame
df = pd.read_sql_query("SELECT * FROM events", conn)

# Optional: Preview the data
print(df.head())

conn.close()

   system_id           checkpoint_id            timestamp   status  \
0       bcis       BCIS_CHECKPOINT_3  2025-05-04T11:17:37  FAILURE   
1        cmf        CMF_CHECKPOINT_2  2025-05-04T11:17:38  SUCCESS   
2    fedebom    FEDEBOM_CHECKPOINT_3  2025-05-04T11:17:38  SUCCESS   
3  reporting  REPORTING_CHECKPOINT_3  2025-05-04T11:17:38  SUCCESS   
4        cmf        CMF_CHECKPOINT_2  2025-05-04T11:17:40  FAILURE   

                         correlation_id         failure_reason  
0  12b35e3e-dc3c-4a5c-83d5-d22496ed75cb  BCIS_FAILURE_REASON_2  
1  b1c165be-901c-451f-86c2-2c97a4148ed2                   None  
2  ca8bf814-27c2-475d-80d4-0ffd6c1eb24e                   None  
3  a57ecd3f-522a-4501-b921-518bc550ccfa                   None  
4  94c95fab-af33-44fc-b389-8f488650c18c                   None  


In [4]:
df.shape

(26, 6)

In [ ]:
# Re-create the events table using the provided schema for demonstration
conn = sqlite3.connect("/mnt/data/traceability.db")
cursor = conn.cursor()

# Drop if exists to avoid duplication
cursor.execute("DROP TABLE IF EXISTS events")

# Create the 'events' table with the provided schema
cursor.execute("""
CREATE TABLE events (
    system_id TEXT,
    checkpoint_id TEXT,
    timestamp TEXT,
    status TEXT,
    correlation_id TEXT,
    failure_reason TEXT
)
""")

conn.commit()

# Now that the schema is in place, attempt to create the spans table again
cursor.execute("DROP TABLE IF EXISTS spans")

cursor.execute("""
CREATE TABLE spans AS
SELECT
    correlation_id,
    system_id,
    MIN(timestamp) AS start_time,
    MAX(timestamp) AS end_time,
    CAST((julianday(MAX(timestamp)) - julianday(MIN(timestamp))) * 86400000 AS INTEGER) AS duration_ms,
    CASE
        WHEN SUM(CASE WHEN status = 'FAILURE' THEN 1 ELSE 0 END) > 0 THEN 'FAILURE'
        ELSE 'SUCCESS'
    END AS status
FROM events
GROUP BY correlation_id, system_id
""")

conn.commit()

# Load spans table for display (will be empty since no event data exists yet)
spans_df = pd.read_sql_query("SELECT * FROM spans ORDER BY correlation_id, system_id", conn)
conn.close()

import ace_tools as tools; tools.display_dataframe_to_user(name="Materialized Spans Table", dataframe=spans_df)
